<a href="https://colab.research.google.com/github/idrissdeme/Projetos-Python-IRD/blob/main/Programa_calculo_Calibracao_LN_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from math import *
import xlrd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
table0 = pd.read_excel('/content/Planilha de calculo de valicao (2).xlsx',sheet_name='Dados_python')
#Vamos criar uma nova dataFrame para evitar mexer nos no dataframe originlamente importados
display(table0) 

table = table0   # criação de uma nova daframe com o nome "tabela" onde os nome das colunas seraão alterada para fim dos cálculos. Assim temos dois dataFrame
# O dataFrame original continua com os mesmos títulos e o dataframe novo tem os nomes de colunas alterados para os cálculos

# renomamos as colunas agora no nosso novo data frame
table.rename(columns = {'q0':'q0', 'Cont. Decaimento (H)':'Lambida_h', 'Cont. Decaimento (s)':'Lambida_s', 'h*f (10)\npSv.cm2':'h_10', 'hpf (10; a)\npSv.cm2':'hp', 'Coeficiente de atenuação \nlinear (cm-1)':'Sec_choque', 'A':'A','Espalhamento':'E' }, inplace = True)
#q0=8;Lambida_h=11; Lambida_s=12; h_10=14; hp=15; Sec_choque=17; A=18; E=19;
display(table)

In [ ]:
fonte=0
for fonte in [0,1,2,3,4,5,6,7,8,9,10,11,12,13]:
  print(fonte, '.','modelo da fonte =',table.iloc[fonte,1], table.iloc[fonte,0])
fonte = int(input("Índice da Fonte: "))
print(fonte)

In [ ]:
grandeza=0
for grandeza in ["0. Ambiente","1. Individual","2. Fluência"]:
  print(grandeza)
grandeza = int(input("Grandeza: "))  
print(grandeza)

In [ ]:
#CÁLCULO DE FLUÊNCIA
d = int(input("Distância (em centimetros): "))
t1 = int(input("Tempo(em segundos):  "))
phi = (table['q0'][fonte]*table['A'][fonte]*table['E'][fonte]*
e**(-table['Sec_choque'][fonte]*d)/(4*pi*d**2))*(1-e**(-table['Lambida_s'][fonte]*
t1))/table['Lambida_s'][fonte]
print('A fluência é:', round(phi), 'n/cm2')

In [ ]:
#CÁLCULO Da TAXA DE FLUÊNCIA
d = int(input("Distância (em centimetros): "))
phi =table['q0'][fonte]*table['A'][fonte]*table['E'][fonte]*e**(-table['Sec_choque'][fonte]*d)/(4*pi*d**2)
print('A taxa de fluência é:', round(phi), 'n/cm^2/s')

Distância (em centimetros): 100
A taxa de fluência é: 82 n/cm^2/s


In [ ]:
#CÁLCULO DO EQUIVALENTE DE DOSE
d = int(input("Distância (em centimetros): "))
t = int(input("Tempo(em horas):  "))
if grandeza==0:
  x='h_10' #ambiente
  y='ambiente'
else:
  x='hp' #individual
  y='individual'
H =(3600*(table[x][fonte]*table['q0'][fonte]*0.000001*
table['A'][fonte]*table['E'][fonte]*
e**(-table['Sec_choque'][fonte]*d))/(4*pi*d**2))*(1-e**(-table['Lambida_h'][fonte]*t))/table['Lambida_h'][fonte]
print('O equivalente de dose',y,'é:', round(H), 'uSv')

In [ ]:
#CÁLCULO DA TAXA DE EQUIVALENTE DE DOSE
d = int(input("Distância: "))
if grandeza==0:
  x='h_10' #ambiente
  y='ambiente'
else:
  x='hp' #individual
  y='individual'
H =(3600*(table[x][fonte]*table['q0'][fonte]*0.000001*
table['A'][fonte]*table['E'][fonte]*e**(-table['Sec_choque'][fonte]*d))/(4*pi*d**2))
print('A taxa de equivalente de dose',y,'é:', round(H), 'uSv/h')

In [ ]:
#CÁLCULO DO TEMPO DE IRRADIAÇÃO
d = int(input("Distância (em centimetros): "))
phi=0
if grandeza==0:
  x='h_10' #ambiente
  y='ambiente'
  Eq_amb = int(input("Equivalente de dose ambiente (em uSv):  "))
  t=-log (1-((Eq_amb*4*pi*d**2*table['Lambida_h'][fonte])/(3600*table[x][fonte]*
  0.000001*table['q0'][fonte]*table['A'][fonte]*table['E'][fonte]*e**((-table['Sec_choque'][fonte]*d)))))/table['Lambida_h'][fonte]
  print ('O tempo de irradiação do equivalente de dose',y,'é:', round(t,2), 'horas')

if grandeza==1:
  x='hp' #individual
  y='individual'
  Eq_ind = int(input("Equivalente de dose individual (em uSv):  "))
  t=-log (1-((Eq_ind*4*pi*d**2*table['Lambida_h'][fonte])/(3600*table[x][fonte]*
  0.000001*table['q0'][fonte]*table['A'][fonte]*table['E'][fonte]*e**((-table['Sec_choque'][fonte]*d)))))/table['Lambida_h'][fonte]
  print ('O tempo de irradiação do equivalente de dose',y,'é:', round(t,2), 'horas')
 
if grandeza==2: #fluencia
  phi = float(input("Fluência (em n/cm^2):  "))
  t=-log (1-((phi*4*pi*d**2*table['Lambida_s'][fonte])/(table['q0'][fonte]*
  table['A'][fonte]*table['E'][fonte]*e**((-table['Sec_choque'][fonte]*d)))))/table['Lambida_s'][fonte]
  print ('O tempo de irradiação da fluência é:', round(t,2), 's')

In [ ]:
#CALCULO DA DISTÂNCIA - Metodo Newton Raphson

#Ambiente
if grandeza==0:
  x='h_10' #ambiente
  Tx_Eq_amb=float(input("Taxa Eq Dose Ambiente (em uSv/h): "))
  distancia=0.0001
  erro=10000
  while erro>=0.00001:
    d0=distancia
    Fd =2*log(distancia)+table['Sec_choque'][fonte]*distancia-log(3600*0.000001*
    table[x][fonte]*table['q0'][fonte]*table['A'][fonte]*table['E'][fonte])*e**(-table['Lambida_h'][fonte])+log(Tx_Eq_amb)+log(4*pi)
    Fd_linha=(2/distancia)+table['Sec_choque'][fonte]
    distancia=distancia-(Fd/Fd_linha)
    erro=distancia-d0
  print('A distância é:', round(distancia,1), 'cm')

#Individual
if grandeza==1:
  x='hp' #individual
  Tx_Eq_ind=float(input("Taxa Eq Dose Individual (em uSv/h): "))
  distancia=0.0001
  erro=10000
  while erro>=0.00001:
    d0=distancia
    Fd =2*log(distancia)+table['Sec_choque'][fonte]*distancia-log(3600*0.000001*
    table[x][fonte]*table['q0'][fonte]*table['A'][fonte]*table['E'][fonte])*e**(-table['Lambida_h'][fonte])+log(Tx_Eq_ind)+log(4*pi)
    Fd_linha=(2/distancia)+table['Sec_choque'][fonte]
    distancia=distancia-(Fd/Fd_linha)
    erro=distancia-d0
  print('A distância é:', round(distancia), 'cm')

#Fluência
if grandeza==2:
  phi=float(input("Taxa fluência (em n/cm2/s): "))
  distancia=0.0001
  erro=10000
  while erro>=0.00001:
    d0=distancia
    Fd =2*log(distancia)+table['Sec_choque'][fonte]*distancia-log(table['q0'][fonte]*
    table['A'][fonte]*table['E'][fonte])*e**(-table['Lambida_s'][fonte])+log(phi)+log(4*pi)
    Fd_linha=(2/distancia)+table['Sec_choque'][fonte]
    distancia=distancia-(Fd/Fd_linha)
    erro=distancia-d0
  print('A distância é:', round(distancia,1), 'cm')